---
# Data Scraping for Voter Turnout Analysis 


### Import all required packages

In [68]:
import requests as rqst
from bs4 import BeautifulSoup as btsup
import pandas as pd
import numpy as np
pd.__version__

'2.2.3'

---
### Wikipedia Website URL for scrapping the 2014 Andhra Pradesh Assemble Elections data

In [69]:
year = input("Please Enter the AP Election year: ")
url = f"https://en.wikipedia.org/wiki/{year}_Andhra_Pradesh_Legislative_Assembly_election#Andhra_Pradesh_results"

---
### Send a HTTP request to the specified URL and save the response

In [70]:
try:
    response = rqst.get(url)
    response.raise_for_status()
    print("Request Successfull")
except rqst.exceptions.ConnectionError as e:
    print("Connection Error: Please check your internet connection.", e)
except rqst.exceptions.RequestException as e:
    print("Request failed:", e)

Request Successfull


---
### Parsing the HTML content to find the table

In [71]:
soup = btsup(response.content, 'html.parser')
soup.prettify()[:200]

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pin'

---
### Extracting the HTML content of the target table

In [72]:
table = soup.find('h3', id='Results_by_constituency', string='Results by constituency').find_next('table')
table.prettify()[:200]

'<table class="wikitable sortable" style="text-align:center;">\n <tbody>\n  <tr>\n   <th rowspan="2">\n    District\n   </th>\n   <th colspan="2">\n    Constituency\n   </th>\n   <th colspan="5">\n    Winner\n   '

---
### Extracting the Rows of the target table

In [73]:
rows = table.find_all('tr')
rows[:5]

[<tr>
 <th rowspan="2">District
 </th>
 <th colspan="2">Constituency
 </th>
 <th colspan="5">Winner
 </th>
 <th colspan="5">Runner Up
 </th>
 <th rowspan="2">Margin
 </th></tr>,
 <tr>
 <th>No.
 </th>
 <th>Name
 </th>
 <th>Candidate
 </th>
 <th colspan="2">Party
 </th>
 <th>Votes
 </th>
 <th>%
 </th>
 <th>Candidate
 </th>
 <th colspan="2">Party
 </th>
 <th>Votes
 </th>
 <th>%
 </th></tr>,
 <tr>
 <td rowspan="8"><a href="/wiki/Srikakulam_district" title="Srikakulam district">Srikakulam</a>
 </td>
 <th>1
 </th>
 <td><a href="/wiki/Ichchapuram_Assembly_constituency" title="Ichchapuram Assembly constituency">Ichchapuram</a>
 </td>
 <td><a href="/wiki/Ashok_Bendalam" title="Ashok Bendalam">Bendalam Ashok</a>
 </td>
 <td data-sort-value="Telugu Desam Party" style="width: 2px; color:inherit; background-color: #FFED00;">
 </td>
 <td scope="row" style="text-align: left;"><a href="/wiki/Telugu_Desam_Party" title="Telugu Desam Party">TDP</a>
 </td>
 <td>110,612
 </td>
 <td>58.58
 </td>
 <td>Piriya

---
### Extracting the Main and Sub Headers of the Table

![Table Header](./table_header.png)

In [74]:
headers = []
for header in rows:
    headers.append(header.find_all('th',string=''))
main_headers = []
sub_headers =  []

for main in headers[0]:
    main_headers.append(main.text.strip())
   
main_headers

for sub in headers[1]:
    sub_headers.append(sub.text.strip()) 

print(main_headers)
print(sub_headers)

['District', 'Constituency', 'Winner', 'Runner Up', 'Margin']
['No.', 'Name', 'Candidate', 'Party', 'Votes', '%', 'Candidate', 'Party', 'Votes', '%']


---
### Data Flattening

- Explanation
    - The above headers contain nested headers.
    - The main headers **<span style='color:lightgreen'>Constituency, Winner, Runner Up</span>** have sub-headers like **<span style='color:green'>No. and Name for Constituency, and Candidate, Party, Votes for Winner and Runner Up</span>**

<style>
    table {
        font-size: 10px; /* Adjust the size as needed */
    }
</style>

### Before Data Flattening of Headers
![Before Data Flattening of Headers](./table_header.png)

### After Data Flattening of Headers

| District    | Constituency No. | Constituency Name | Winner Candidate | Winner Party | Winner Votes | Runner-Up Candidate | Runner-Up Party | Runner-Up Votes | Margin |
|-------------|------------------|-------------------|------------------|--------------|--------------|---------------------|-----------------|-----------------|--------|
| District A  | 1                | Name A            | Candidate A      | Party A      | 12000        | Candidate B         | Party B         | 11500           | 500    |
| District B  | 2                | Name B            | Candidate C      | Party C      | 15000        | Candidate D         | Party D         | 14000           | 1000   |
| District C  | 3                | Name C            | Candidate E      | Party E      | 18000        | Candidate F         | Party F         | 17500           | 500    |



In [75]:
flattened_headers = []


for header in main_headers:
    if header == 'District' or header == 'Margin':
        # Add directly for single columns
        flattened_headers.append(header)
    elif header == 'Constituency':
        # Map 'Constituency' to 'No.' and 'Name'
        flattened_headers.extend(['Constituency No.', 'Constituency Name'])
    elif header == 'Winner':
        # Map 'Winner' to 'Candidate', 'Party', 'Votes'
        flattened_headers.extend(['Winner Candidate', 'Winner Party', 'Winner Votes'])
    elif header == 'Runner Up':
        # Map 'Runner Up' to 'Candidate', 'Party', 'Votes'
        flattened_headers.extend(['Runner-Up Candidate', 'Runner-Up Party', 'Runner-Up Votes'])

flattened_headers.append('Link')
print(flattened_headers)
len(flattened_headers)

['District', 'Constituency No.', 'Constituency Name', 'Winner Candidate', 'Winner Party', 'Winner Votes', 'Runner-Up Candidate', 'Runner-Up Party', 'Runner-Up Votes', 'Margin', 'Link']


11

---
### Extract the Data of the Target Table

In [76]:
table_data = []
for row in rows[2:]:
    table_data.append([col.text.strip() for col in row.find_all(['td','th']) if len(col.text.strip())!=0])
table_data[:5]

[['Srikakulam',
  '1',
  'Ichchapuram',
  'Bendalam Ashok',
  'TDP',
  '110,612',
  '58.58',
  'Piriya Vijaya',
  'YSRCP',
  '70,829',
  '37.51',
  '39,783'],
 ['2',
  'Palasa',
  'Gouthu Sireesha',
  'TDP',
  '101,560',
  '60.44',
  'Seediri Appalaraju',
  'YSRCP',
  '61,210',
  '36.43',
  '40,350'],
 ['3',
  'Tekkali',
  'Kinjarapu Atchannaidu',
  'TDP',
  '107,923',
  '55.71',
  'Duvvada Srinivas',
  'YSRCP',
  '73,488',
  '37.94',
  '34,435'],
 ['4',
  'Pathapatnam',
  'Mamidi Govinda Rao',
  'TDP',
  '89,452',
  '54.76',
  'Reddy Shanthi',
  'YSRCP',
  '62,925',
  '38.52',
  '26,527'],
 ['5',
  'Srikakulam',
  'Gondu Shankar',
  'TDP',
  '117,091',
  '60.93',
  'Dharmana Prasada Rao',
  'YSRCP',
  '64,570',
  '33.60',
  '52,521']]

---
### 1. Add Missing District Data

### 2. Add Wikipedia Links for Each Constituency

###### Don't Run this cell multiple times if need run from previous cell

In [77]:
[data if len(data)==10 or len(data)==12  else data.insert(0,table_data[index-1][0]) for index, data in enumerate(table_data)] 
[data.append(f'https://en.wikipedia.org/wiki/{data[2]}_Assembly_constituency') for data in table_data]
table_data[:5]

[['Srikakulam',
  '1',
  'Ichchapuram',
  'Bendalam Ashok',
  'TDP',
  '110,612',
  '58.58',
  'Piriya Vijaya',
  'YSRCP',
  '70,829',
  '37.51',
  '39,783',
  'https://en.wikipedia.org/wiki/Ichchapuram_Assembly_constituency'],
 ['Srikakulam',
  '2',
  'Palasa',
  'Gouthu Sireesha',
  'TDP',
  '101,560',
  '60.44',
  'Seediri Appalaraju',
  'YSRCP',
  '61,210',
  '36.43',
  '40,350',
  'https://en.wikipedia.org/wiki/Palasa_Assembly_constituency'],
 ['Srikakulam',
  '3',
  'Tekkali',
  'Kinjarapu Atchannaidu',
  'TDP',
  '107,923',
  '55.71',
  'Duvvada Srinivas',
  'YSRCP',
  '73,488',
  '37.94',
  '34,435',
  'https://en.wikipedia.org/wiki/Tekkali_Assembly_constituency'],
 ['Srikakulam',
  '4',
  'Pathapatnam',
  'Mamidi Govinda Rao',
  'TDP',
  '89,452',
  '54.76',
  'Reddy Shanthi',
  'YSRCP',
  '62,925',
  '38.52',
  '26,527',
  'https://en.wikipedia.org/wiki/Pathapatnam_Assembly_constituency'],
 ['Srikakulam',
  '5',
  'Srikakulam',
  'Gondu Shankar',
  'TDP',
  '117,091',
  '60.9

---
### Converting table_data into Pandas DataFrame 

In [78]:
if(len(table_data[0])==len(flattened_headers)+2):
    flattened_headers.insert(6,'%')
    flattened_headers.insert(10,'%')
df = pd.DataFrame(data=table_data, columns=flattened_headers)
df

,District,Constituency No.,Constituency Name,Winner Candidate,Winner Party,Winner Votes,%,Runner-Up Candidate,Runner-Up Party,Runner-Up Votes,%,Margin,Link
0,Srikakulam,1,Ichchapuram,Bendalam Ashok,TDP,"110,612",58.58,Piriya Vijaya,YSRCP,"70,829",37.51,"39,783",https://en.wikipedia.org/wiki/Ichchapuram_Asse...
1,Srikakulam,2,Palasa,Gouthu Sireesha,TDP,"101,560",60.44,Seediri Appalaraju,YSRCP,"61,210",36.43,"40,350",https://en.wikipedia.org/wiki/Palasa_Assembly_...
2,Srikakulam,3,Tekkali,Kinjarapu Atchannaidu,TDP,"107,923",55.71,Duvvada Srinivas,YSRCP,"73,488",37.94,"34,435",https://en.wikipedia.org/wiki/Tekkali_Assembly...
3,Srikakulam,4,Pathapatnam,Mamidi Govinda Rao,TDP,"89,452",54.76,Reddy Shanthi,YSRCP,"62,925",38.52,"26,527",https://en.wikipedia.org/wiki/Pathapatnam_Asse...
4,Srikakulam,5,Srikakulam,Gondu Shankar,TDP,"117,091",60.93,Dharmana Prasada Rao,YSRCP,"64,570",33.60,"52,521",https://en.wikipedia.org/wiki/Srikakulam_Assem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,Chittoor,171,Gangadhara Nellore (SC),V. M. Thomas,TDP,"101,176",55.22,Kalattur Krupa Lakshmi,YSRCP,"75,165",41.02,"26,011",https://en.wikipedia.org/wiki/Gangadhara Nello...
171,Chittoor,172,Chittoor,Gurajala Jagan Mohan,TDP,"88,066",52.49,M. Vijayananda Reddy,YSRCP,"73,462",43.78,"14,604",https://en.wikipedia.org/wiki/Chittoor_Assembl...
172,Chittoor,173,Puthalapattu (SC),Kalikiri Murali Mohan,TDP,"102,137",50.99,M. Sunil Kumar,YSRCP,"86,503",43.20,"15,634",https://en.wikipedia.org/wiki/Puthalapattu (SC...
173,Chittoor,174,Palamaner,N. Amarnath Reddy,TDP,"123,232",52.09,N. Venkate Gowda,YSRCP,"103,110",43.59,"20,122",https://en.wikipedia.org/wiki/Palamaner_Assemb...


---
### Converting DataFrame into Excel Sheet

In [79]:
df.to_excel(f'ap_ac_{year}.xlsx',sheet_name= f'{year}',index=False)